In [10]:
import tvm
import tvm.ir
from tvm import relay
from tvm.relay import transform
from tvm.relay.dataflow_pattern import wildcard, is_op
from tvm.relay.op.contrib import register_pattern_table
from tvm.relay.testing import byoc
import numpy as np

In [18]:
def example():
    data_dtype = "int32"
    axis = 0
    x_data = np.arange(-32, 32, 1).reshape(1, 64).astype(data_dtype)
    y_data = np.arange(-64, 64, 2).reshape(1, 64).astype(data_dtype)

    x_scale = relay.const((62 + 64) / (np.power(2, 32) - 1.0), "float32")
    y_scale = relay.const((62 + 64) / (np.power(2, 32) - 1.0), "float32")
    x_zero_point = relay.const(0, "int32")
    y_zero_point = relay.const(0, "int32")

    x = relay.var("x", shape=(1, 64), dtype=data_dtype)
    y = relay.var("y", shape=(1, 64), dtype=data_dtype)
    z = relay.qnn.op.concatenate(
        (x, y),
        input_scales=(x_scale, y_scale),
        input_zero_points=(x_zero_point, y_zero_point),
        output_scale=y_scale,
        output_zero_point=relay.const(1, "int32"),
        axis=axis,
    )

    func = relay.Function([x, y], z)
    mod = tvm.IRModule.from_expr(func)

mod = example()

TVMError: Traceback (most recent call last):
  5: TVMFuncCall
  4: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>::AssignTypedLambda<tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&)>(tvm::IRModule (*)(tvm::RelayExpr const&, tvm::runtime::Map<tvm::GlobalVar, tvm::BaseFunc, void, void> const&, tvm::runtime::Map<tvm::GlobalTypeVar, tvm::TypeData, void, void> const&), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  3: tvm::IRModule

In [4]:
def _register_external_op_helper(op_name, supported=True):
    @tvm.ir.register_op_attr(op_name, "target.cuda")
    def _func_wrapper(attrs, args):
        return supported
    return _func_wrapper


In [6]:
_register_external_op_helper("nn.conv2d")
_register_external_op_helper("nn.relu")
_register_external_op_helper("add")

<function __main__._register_external_op_helper.<locals>._func_wrapper(attrs, args)>

In [7]:
def make_pattern(with_bias=True):
    data = wildcard()
    weight = wildcard()
    bias = wildcard()
    conv = is_op('nn.conv2d')(data, weight)
    if with_bias:
        conv_out = is_op('add')(conv, bias)
    else:
        conv_out = conv
    return is_op('nn.relu')(conv_out)

@register_pattern_table("cuda")
def pattern_table():
    conv2d_bias_relu_pat = ("cuda.conv2d_bias_relu", make_pattern(with_bias=True))
    conv2d_relu_pat = ("cuda.conv2d_relu", make_pattern(with_bias=False))
    cuda_patterns = [conv2d_bias_relu_pat, conv2d_relu_pat]
    return cuda_patterns


In [19]:
mod = transform.MergeComposite(pattern_table)(mod)
mod = transform.AnnotateTarget(["cuda"])(mod) # Output: Figure 2
mod = transform.MergeCompilerRegions()(mod) # Output: Figure 3
mod = transform.PartitionGraph()(mod) # Output: Figure 4

TypeError: 'function' object is not iterable